In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
from scipy.spatial import Voronoi

In [2]:
#### Inputs

# damage results from Analysis-1
dam_results = pd.read_excel('ci_1.01_0.25_mosek_10000_1.xlsx', index_col = [0])

In [3]:
print(f'Total number of substations {len(dam_results)}')
print(f'Number of flooded substations {len(dam_results[dam_results['depth'] > 0])}')
print(f'Number of failed substations {len(dam_results[dam_results['flood_fail'] == 0])}')

Total number of substations 42
Number of flooded substations 15
Number of failed substations 7


###### These 7 substations are critical and selected for adaptation. But with limited capital how do we select or priortise the substaions. Example: The infrastructrue owner has only funds to fix two substations which ones should be them.

In [4]:
failed_ss = dam_results[dam_results['flood_fail'] == 0]
failed_ss = failed_ss.reset_index(drop = True)

In [5]:
# an empty df to store the sequences
sequence_df = pd.DataFrame(index = np.arange(0, len(failed_ss), 1))

# Strategy 1: Fix the substation that is severly flooded first

In [6]:
def mergeing_gdf(failed_ss):
    stat1 = failed_ss
    ss_nl = gpd.read_parquet('substaions_nl33.parquet')
    stat1 = stat1.drop(columns = ['geometry', 'voltage_kv'])
    merged_gdf = stat1.merge(ss_nl, on='osmid', how='inner')
    merged_gdff = gpd.GeoDataFrame(merged_gdf, geometry='geometry')
    return merged_gdff

In [7]:
gdf_depth = mergeing_gdf(failed_ss) 
gdf_depth = gdf_depth.sort_values(by=['depth'], ascending= False)
gdf_depth = gdf_depth.reset_index(drop = True)

In [8]:
gdf_depth

,osmid,depth,flood_fail,voltage_kv,geometry
0,165250724,3.5617,0,23.0,"POLYGON ((4.52995 51.95506, 4.529 51.95491, 4...."
1,165242801,3.1665,0,50.0,"POLYGON ((4.62897 51.91306, 4.6289 51.913, 4.6..."
2,171267263,2.4107,0,50.0,"POLYGON ((4.14748 51.89278, 4.14776 51.89217, ..."
3,198335271,2.0060,0,23.0,"POLYGON ((4.36261 51.87452, 4.36249 51.8745, 4..."
4,72888152,1.7750,0,150.0,"POLYGON ((4.52803 52.03733, 4.53238 52.03657, ..."
5,23135620,1.6894,0,50.0,"POLYGON ((4.2026 51.7474, 4.20405 51.7465, 4.2..."
6,1266908852,1.4831,0,10.0,"POLYGON ((4.50367 51.87817, 4.50426 51.87779, ..."


In [9]:
sequence_df['depth'] = gdf_depth['osmid']

# Strategy 2: Fix the substations that has more proximity to the industrial areas - sector non specific

In [10]:
def poly_to_pt(gdf):
    
    gdf['centroid'] = gdf.geometry.centroid
    gdf = gdf.drop(columns='geometry')
    gdf = gdf.rename(columns={'centroid': 'geometry'})
    gdf = gdf.set_geometry('geometry')
    return gdf

In [11]:
def filter_poly(gdf_polygons, gdf_points):
    join = gpd.sjoin(gdf_polygons, gdf_points, predicate='contains')
    gdf_polygons_filtered = gdf_polygons.loc[gdf_polygons.index.isin(join.index)]
    gdf_polygons_filtered = gdf_polygons_filtered.reset_index(drop = True)
    return gdf_polygons_filtered 

In [12]:
def nearest_ss(gdf_industries , gdf_substations):
    gdf_industries = gdf_industries.to_crs(gdf_substations.crs)
    gdf_joined = gpd.sjoin_nearest(gdf_industries, gdf_substations[['osmidss', 'geometry']], how="left", distance_col="distance_to_ss")
    return gdf_joined

In [13]:
# Industrial sites from OSM
ind_nl33 = gpd.read_parquet('is_nl33.parquet')

#### Fetched results from places API
industrial = gpd.read_parquet('industrial.parquet')

# SUbstations in NL33
ss_nl33 = gpd.read_parquet('substaions_nl33.parquet')
ss_nl33 = poly_to_pt(ss_nl33)
ss_nl33 = ss_nl33.rename(columns = {'osmid': 'osmidss'})

C:\Users\sva100\AppData\Local\Temp\ipykernel_35688\699070099.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid


In [14]:
industrial = poly_to_pt(industrial)
ind_nl33_filt  = filter_poly(ind_nl33, industrial)  #Filtering the osm sites that are hosting the fetched indutries only

# Finding the nearest substation to the industrial sites
ind_nl33_filt = nearest_ss(ind_nl33_filt , ss_nl33)

ind_nl33_filt['area'] = ind_nl33_filt.geometry.area
gdf_ssind = poly_to_pt(gdf_depth)

for i in range(len(gdf_ssind)):
    osmid_ss = gdf_ssind.loc[i, 'osmid']
    ind_df1 = ind_nl33_filt[ind_nl33_filt['osmidss'] == osmid_ss]
    gdf_ssind.loc[i, 'area_ind'] = ind_df1['area'].sum()

gdf_ssind = gdf_ssind.sort_values(by = 'area_ind', ascending = False)
gdf_ssind = gdf_ssind.reset_index(drop = True)
sequence_df['area_ind'] = gdf_ssind['osmid']

C:\Users\sva100\AppData\Local\Temp\ipykernel_35688\699070099.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid
C:\Users\sva100\AppData\Local\miniforge3\envs\mria\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
C:\Users\sva100\AppData\Local\Temp\ipykernel_35688\1004379700.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ind_nl33_filt['area'] = ind_nl33_filt.geometry.area
C:\Users\sva100\AppData\Local\Temp\ipykernel_35688\699070099.py:3: UserWarning: Geometry is in a geogra

# Strategy 3: Fix the substations that has more proximity to the industrial areas - serving C19 and C20 sectors

In [15]:
industrial_sector = industrial[industrial['Sector_id'].isin(['C19', 'C20'])]

In [16]:
industrial_sector = industrial_sector.reset_index(drop = True)

In [17]:
industrial_sector = poly_to_pt(industrial_sector)
ind_nl33_filt_sector  = filter_poly(ind_nl33, industrial_sector)  #Filtering the osm sites that are hosting only C19 and C20 industries


C:\Users\sva100\AppData\Local\Temp\ipykernel_35688\699070099.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid


In [18]:
# Finding the nearest substation to the industrial sites
ind_nl33_filt_sector = nearest_ss(ind_nl33_filt_sector , ss_nl33)

C:\Users\sva100\AppData\Local\miniforge3\envs\mria\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [19]:
ind_nl33_filt_sector['area'] = ind_nl33_filt_sector.geometry.area
gdf_sector = poly_to_pt(gdf_depth)

C:\Users\sva100\AppData\Local\Temp\ipykernel_35688\3891634995.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ind_nl33_filt_sector['area'] = ind_nl33_filt_sector.geometry.area
C:\Users\sva100\AppData\Local\Temp\ipykernel_35688\699070099.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid


In [20]:
for i in range(len(gdf_sector)):
    osmid_ss = gdf_sector.loc[i, 'osmid']
    ind_df1 = ind_nl33_filt_sector[ind_nl33_filt_sector['osmidss'] == osmid_ss]
    gdf_sector.loc[i, 'area_sector'] = ind_df1['area'].sum()


gdf_sector = gdf_sector.sort_values(by = 'area_sector', ascending = False)
gdf_sector = gdf_sector.reset_index(drop = True)

In [25]:
gdf_ssind

,osmid,depth,flood_fail,voltage_kv,geometry,area_ind
0,1266908852,1.4831,0,10.0,POINT (4.50367 51.87781),0.000575
1,72888152,1.7750,0,150.0,POINT (4.53038 52.03622),0.000399
2,198335271,2.0060,0,23.0,POINT (4.36232 51.8748),0.000322
3,165250724,3.5617,0,23.0,POINT (4.5294 51.95532),0.000315
4,165242801,3.1665,0,50.0,POINT (4.62862 51.91244),0.000312
5,171267263,2.4107,0,50.0,POINT (4.14736 51.89242),0.000209
6,23135620,1.6894,0,50.0,POINT (4.20293 51.74671),0.000091


In [24]:
gdf_sector

,osmid,depth,flood_fail,voltage_kv,geometry,area_sector
0,198335271,2.0060,0,23.0,POINT (4.36232 51.8748),0.000295
1,1266908852,1.4831,0,10.0,POINT (4.50367 51.87781),0.000276
2,165250724,3.5617,0,23.0,POINT (4.5294 51.95532),0.000127
3,165242801,3.1665,0,50.0,POINT (4.62862 51.91244),0.000116
4,171267263,2.4107,0,50.0,POINT (4.14736 51.89242),0.000040
5,23135620,1.6894,0,50.0,POINT (4.20293 51.74671),0.000027
6,72888152,1.7750,0,150.0,POINT (4.53038 52.03622),0.000005


In [21]:
sequence_df['area_sector'] = gdf_sector['osmid']

In [22]:
sequence_df

,depth,area_ind,area_sector
0,165250724,1266908852,198335271
1,165242801,72888152,1266908852
2,171267263,198335271,165250724
3,198335271,165250724,165242801
4,72888152,165242801,171267263
5,23135620,171267263,23135620
6,1266908852,23135620,72888152


In [23]:
sequence_df.to_parquet('sequence.parquet')